# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [ ]:
# Your code goes here
import pandas as pd

# Dataset URL = https://bit.ly/3BPcobU
#

# Open the dataset in chunks with a chunk size of 5000 rows

df_chunk =  pd.read_csv("https://bit.ly/3BPcobU", chunksize=5000)


# Process each chunk
for chunk in chunks:
    # Get some information about the chunk
    print("Shape:", chunk.shape)  # Prints the number of rows and columns in the chunk
    print("Columns:", chunk.columns)  # Prints the column names
    print("\n")  # Adds a blank line between chunks
    # Perform any other operations you need to on the chunk
    # ...




In [ ]:
import pandas as pd

# Read the data into a dataframe
df = pd.read_csv("https://bit.ly/3H2XVgC")

# Find the rows with missing values for each column
null_counts = df.isnull().sum()

# Print the results
print(null_counts)



In [ ]:
import pandas as pd

# Read the data into a dataframe
df = pd.read_csv("https://bit.ly/3H2XVgC")

# Get the memory usage of each column
memory_footprint = df.memory_usage()

# Print the results
print(memory_footprint)



In [ ]:
import pandas as pd

# Open the dataset in chunks with a chunk size of 5000 rows
chunks = pd.read_csv("https://bit.ly/3H2XVgC", chunksize=5000)

# Initialize a variable to hold the total memory usage
total_memory = 0

# Process each chunk
for chunk in chunks:
    # Get the memory usage of the chunk
    chunk_memory = chunk.memory_usage().sum()
    # Add the chunk's memory usage to the total
    total_memory += chunk_memory

# Print the total memory usage
print("Total memory usage:", total_memory, "bytes")



In [ ]:
import pandas as pd

# Read the data into a dataframe
df = pd.read_csv("https://bit.ly/3H2XVgC")

# Calculate the percentage of missing values in each column
missing_values_percent = df.isnull().mean()

# Drop columns that have more than 50% missing values
df = df.drop(columns=missing_values_percent[missing_values_percent > 0.5].index)



## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [ ]:
# Your code goes here
import pandas as pd

df = pd.read_csv("https://bit.ly/3H2XVgC")
print(df.dtypes)



In [ ]:
import pandas as pd

df = pd.read_csv("https://bit.ly/3H2XVgC")
text_columns = df.select_dtypes(include=['object'])

for column in text_columns:
    print(f'Column {column}: {text_columns[column].dtype}')



In [ ]:
import pandas as pd

df = pd.read_csv("https://bit.ly/3H2XVgC")
unique_counts = df.nunique()
print(unique_counts)



In [ ]:
import pandas as pd

df = pd.read_csv("https://bit.ly/3H2XVgC")
df_dummies = pd.get_dummies(df, columns=['text_column'])



In [ ]:
import pandas as pd

df = pd.read_csv("https://bit.ly/3H2XVgC")
memory_usage = df.memory_usage(deep=True)

# Select columns that are consuming a large amount of memory
high_memory_columns = memory_usage[memory_usage > 10_000_000].index

# Convert high memory columns to categorical data type
df[high_memory_columns] = df[high_memory_columns].astype('category')

df_dummies = pd.get_dummies(df, columns=['text_column'])
df_cleaned = pd.concat([df, df_dummies], axis=1)

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [ ]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('database.db')

# Load the data into the table
df.to_sql('table_name', conn, if_exists='replace', index=False)

# Close the connection
conn.close()



In [ ]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('database.db')

# Read the data in chunks
chunks = pd.read_csv("https://bit.ly/3H2XVgC", chunksize=1000)

# Iterate over the chunks
for i, chunk in enumerate(chunks):
    # Load the chunk into a new table in the database
    chunk.to_sql(f'table_{i}', conn, if_exists='replace', index=False)

# Close the connection
conn.close()



In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('database.db')

# Query the table and retrieve the column information
cursor = conn.cursor()
cursor.execute("SELECT * FROM PRAGMA table_3('table_3')")
columns = cursor.fetchall()

# Print the column information
for column in columns:
    print(f'Column name: {column[1]}, data type: {column[2]}')

# Close the connection
conn.close()



## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [ ]:
# Calculate the investment amount at the 90th and 99th percentiles
top_10_threshold = df['raised_amount_usd'].quantile(0.90)
top_1_threshold = df['raised_amount_usd'].quantile(0.99)

# Calculate the total investment amount
total_investment = df['raised_amount_usd'].sum()

# Calculate the proportion of funds raised by the top 10% of startups
top_10_proportion = df[df['raised_amount_usd'] >= top_10_threshold]['raised_amount_usd'].sum() / total_investment
print(f'The top 10% of startups raised {top_10_proportion:.2%} of the total funds.')

# Calculate the proportion of funds raised by the top 1% of startups
top_1_proportion = df[df['raised_amount_usd'] >= top_1_threshold]['raised_amount_usd'].sum() / total_investment
print(f'The top 1% of startups raised {top_1_proportion:.2%} of the total funds.')


The top 10% of startups raised 51.22% of the total funds.
The top 1% of startups raised 19.35% of the total funds.


In [ ]:
import pandas as pd

# Load data into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM table_name", conn)

# Print the column names
print(df.columns)


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('database.db')

# Create a cursor
cursor = conn.cursor()

# Execute a SELECT statement to retrieve the names of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the rows from the SELECT statement
tables = cursor.fetchall()

# Print the names of all tables
for table in tables:
    print(table[0])

# Close the cursor and connection
cursor.close()
conn.close()


In [ ]:
# Group the data by category and calculate the total investment amount for each category
category_investments = df.groupby('company_category_code')['raised_amount_usd'].sum()


# Find the categories with the maximum total investment amount
most_invested_categories = category_investments[category_investments == category_investments.max()]

print(f'The category that attracted the most investments was {most_invested_categories}.')




In [ ]:
# Group the data by investor and calculate the total investment amount for each investor
investor_investments = df.groupby('investor')['amount'].sum()

# Find the investor with the maximum total investment amount
top_investor = investor_investments.idxmax()


print(f'The investor who contributed the most money was {top_investor}.')




In [ ]:
import sqlite3
conn = sqlite3.connect("startup_investments.db")
cur = conn.cursor()

import pandas as pd
df = pd.read_sql_query("SELECT * FROM startup", conn)

investor_groups = df.groupby("investor_name")
investment_totals = investor_groups["investment_amount"].sum()

investment_totals = investment_totals.sort_values(ascending=False)

top_investor = investment_totals.index[0]
print(f"The top investor is {top_investor} with a total investment of ${investment_totals[0]:.2f}.")




In [ ]:
import sqlite3
conn = sqlite3.connect("startup_investments.db")
cur = conn.cursor()

import pandas as pd
df = pd.read_sql_query("SELECT * FROM investments", conn)

round_groups = df.groupby("funding_round")
investment_counts = round_groups.size()

investment_counts = investment_counts.sort_values(ascending=False)


most_popular = investment_counts.index[0]
least_popular = investment_counts.index[-1]
print(f"The most popular funding round is {most_popular} with {investment_counts[0]} investments.")
print(f"The least popular funding round is {least_popular} with {investment_counts[-1]} investments.")




In [122]:
#Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).

import sqlite3
conn = sqlite3.connect("startup_investments.db")
cur = conn.cursor()

cur.execute("SELECT investor_name, funding_round, investment_amount FROM investments")
data = cur.fetchall()

import pandas as pd
df = pd.DataFrame(data, columns=["investor_name", "funding_round", "investment_amount"])


cur.close()
conn.close()


